<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/20250222_triangular_mm_kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA Triangular Matrix Multiplication Benchmark

## Purpose

This project aims to compare the performance of various CUDA kernel implementations for lower triangular matrix multiplication with PyTorch's built-in functions and to investigate a suspected "memory stealing" optimization technique.

## Background

A CUDA code (referred to as the "Sakana" version) published by a Japanese unicorn company was found to be unusually fast, but with incorrect calculation results.  It was suspected that this code might be "stealing" results from PyTorch's calculations by omitting memory initialization and CUDA kernel synchronization.

This repository compares and validates the following CUDA kernel implementations and PyTorch's built-in functions:

*   **Sakana:** The original, suspicious code (1D grid, no zero-initialization, no synchronization).
*   **Improved:** A corrected version of the Sakana code, made to work correctly (2D grid, zero-initialization, synchronization).
*   **Gemini:** A version attempting different optimizations (2D grid, zero-initialization, synchronization).
*   **Gemini2:** Gemini version with shared memory and Cooperative Groups optimizations.
*   **Gemini3:** A version modified to intentionally perform "memory stealing" (for comparative verification).
*   **GeminiMMA:** Optimized using Warp Matrix Multiply-Accumulate (WMMA).
*   **PyTorch:** Implementation using PyTorch's built-in functions (`torch.matmul` and `.tril()`).

## Validation Methodology

1.  **Correctness Validation (`allclose`):**
    *   The `torch.allclose` function is used to check if the calculation results of each CUDA kernel and PyTorch match within a certain tolerance.
    *   The execution order of the CUDA kernel and PyTorch is swapped to verify that the results do not change ("memory stealing" check).

2.  **Performance Measurement (`do_bench`):**
    *   The `triton.testing.do_bench` function is used to measure the execution time of each implementation.
    *   The average execution time over multiple runs is compared.

## Usage

1.  **Environment:**
    *   A GPU with CUDA support (NVIDIA Tesla T4 recommended)
    *   Python 3.11
    *   PyTorch (2.x or higher, CUDA-enabled version)
    *   Required packages: `triton`, `ninja`, `setuptools`

        ```bash
        pip install --upgrade --force-reinstall torch torchvision torchaudio
        pip install triton ninja setuptools
        ```
    *   (Note) In a Colab environment, errors may occur during the installation of `torch`. If this happens, try restarting the runtime and installing `torch` without specifying `--index-url`.

2.  **Execution:**
    *   Copy and paste the Python code from this repository into a Google Colab code cell.
    *   Run the code cell.

## Results

| Version        | CUDA first | Torch first | allclose | Execution Time (ms) | Notes                                      |
| :------------- | :--------- | :---------- | :------- | -----------------: | :----------------------------------------- |
| Improved       | True       | True        | True     |     (measurement)     | Works correctly                            |
| Gemini         | True       | True        | True     |       (measurement)    | Works correctly                            |
| Gemini2        |      |      |         |     (measurement)      | Bug, or optimization unsuitable for GPU/problem |
| Gemini3        | False      | False       | False    | (measurement)  | Intentional cheating (memory stealing)   |
| GeminiMMA      |            |            |          | (measurement)       | Optimized using WMMA                        |
| Sakana         | False      | True        | -       | (measurement)      | Unfair speedup due to "memory stealing"    |
| PyTorch        | -           | -        |  -   |   (measurement)        |                                            |

*   **Sakana:** `allclose` is `True` only when PyTorch is executed first, confirming "memory stealing."
*   **Gemini3:** Intentionally incorrect code, `allclose` is always `False`.
* **Gemini2:** Implemented optimization using shared memory and cooperative groups
* **GeminiMMA**: Implemented optimization with WMMA. Results and performance can significantly change depending on the environment.
*   **Improved, Gemini:** Work correctly, and `allclose` is always `True`.
*   **PyTorch:** In many cases, PyTorch's built-in functions were faster than manually optimized CUDA kernels. This is likely because PyTorch uses highly optimized libraries like cuBLAS internally.

## Execution Environment

*   Python: (version)
*   OS: (version)
*   CUDA: (version)
*   PyTorch: (version)
*   cuDNN: (version)
*   GPU: (model name, e.g., NVIDIA Tesla T4)
*   (Output of `nvidia-smi`)

## Conclusion

This investigation confirmed that the original Sakana CUDA code has serious issues and achieves unfair speedups through "memory stealing."  It was also found that PyTorch's built-in functions are often faster than manually optimized CUDA kernels in many cases.

CUDA kernel optimization depends heavily on the GPU architecture, the characteristics of the problem, and a deep understanding of CUDA programming.  Simply rewriting code does not always improve performance, and in some cases, it may be better to rely on highly optimized libraries like PyTorch.

# CUDA Triangular Matrix Multiplication Benchmark

## 目的

このプロジェクトは、下三角行列乗算 (Triangular Matrix Multiplication) のための വിവിധなCUDAカーネル実装とPyTorchの組み込み関数とのパフォーマンス比較、および「盗み見」と呼ばれる不正な高速化手法の検証を目的としています。

## 背景

ある日本のユニコーン企業が公開したCUDAコード (Sakanaバージョンと呼ぶ) が、異常に高速であるにもかかわらず、計算結果が不正であるという疑惑が持ち上がりました。このコードは、メモリの初期化を省略し、CUDAカーネルの同期を行わないことで、PyTorchの計算結果を「盗み見」している可能性が指摘されました。

このリポジトリでは、以下のCUDAカーネル実装とPyTorchの組み込み関数を比較検証します。

*   **Sakana:** オリジナルの疑惑のコード (1次元グリッド、ゼロ初期化なし、同期なし)。
*   **Improved:** Sakanaバージョンを修正し、正しく動作するようにしたバージョン (2次元グリッド、ゼロ初期化、同期あり)。
*   **Gemini:** 別の最適化を試みたバージョン (2次元グリッド、ゼロ初期化、同期あり)。
*   **Gemini2:** Geminiバージョンに共有メモリとCooperative Groupsを使った最適化を追加したバージョン。
*   **Gemini3:** 意図的に「盗み見」を行うように改変したバージョン (比較検証用)。
*    **GeminiMMA:** Warp Matrix Multiply-Accumulate (WMMA) を使って最適化。
*   **PyTorch:** PyTorchの組み込み関数 (`torch.matmul` と `.tril()`) を使用した実装。

## 検証方法

1.  **正当性検証 (`allclose`):**
    *   各CUDAカーネルとPyTorchの計算結果が、ある許容誤差内で一致するかどうかを `torch.allclose` 関数で確認します。
    *   CUDAカーネルの実行順序とPyTorchの実行順序を入れ替えて、結果が変わらないかを確認します (「盗み見」検証)。

2.  **パフォーマンス測定 (`do_bench`):**
    *   `triton.testing.do_bench` 関数を使用して、各実装の実行時間を計測します。
    *   複数回の実行時間の平均値を比較します。

## 使用方法

1.  **環境:**
    *   CUDAが利用可能なGPU (NVIDIA Tesla T4を推奨)
    *   Python 3.11
    *   PyTorch (2.x以上、CUDA対応版)
    *   必要なパッケージ: `triton`, `ninja`, `setuptools`

        ```bash
        pip install --upgrade --force-reinstall torch torchvision torchaudio
        pip install triton ninja setuptools
        ```
    *   (注意) Colab環境では、`torch`のインストールでエラーが発生することがあります。その場合は、ランタイムを再起動し、`--index-url` を指定せずに `pip install torch`を実行する、などの対応が必要です。

2.  **実行:**
    *   このリポジトリのPythonコードを、Google Colabのコードセルにコピー＆ペーストします。
    *   コードセルを実行します。

## 結果

| バージョン     | CUDA first | Torch first | allclose | 実行時間 (ms) | 備考                               |
| :------------- | :--------- | :---------- | :------- | -------------: | :--------------------------------- |
| Improved       | True       | True        | True     |     (計測結果)     | 正しく動作                         |
| Gemini         | True       | True        | True     |       (計測結果)        | 正しく動作                         |
| Gemini2        |      |      |         |     (計測結果)          |       バグまたは、最適化がGPU/問題設定に合わない                             |
| Gemini3      | False      | False       | False   | (計測結果)      | 意図的な不正 (盗み見)               |
| GeminiMMA     |       |      |         |       (計測結果)        |       WMMAによる最適化                             |
| Sakana         | False      | True        | -       | (計測結果)      | 「盗み見」による不正な高速化       |
| PyTorch        | -           | -        |  -   |   (計測結果)   |                                    |

*   **Sakana:** PyTorchを先に実行した場合にのみ `allclose` が `True` になり、「盗み見」が確認されました。
*   **Gemini3:** 意図的に不正なコードにしたもので、常に`allclose` が `False`。
*   **Gemini2:**  共有メモリとCooperative Groupsを使った最適化を実装。
*    **GeminiMMA:** WMMAを使った最適化を実装。 結果とパフォーマンスは実行環境によって大きく変わる可能性があります。
*   **Improved, Gemini:** 正しく動作し、`allclose` は常に `True` です。
*   **PyTorch:** 多くのケースで、PyTorchの組み込み関数が、手動で最適化したCUDAカーネルよりも高速でした。これは、PyTorchが内部でcuBLASなどの高度に最適化されたライブラリを使用しているためと考えられます。

## 実行環境

*   Python: (バージョン)
*   OS: (バージョン)
*   CUDA: (バージョン)
*   PyTorch: (バージョン)
*   cuDNN: (バージョン)
*   GPU: (モデル名, 例: NVIDIA Tesla T4)
*   (nvidia-smi の出力)

## 結論

今回の検証により、オリジナルのSakana CUDAコードには重大な問題があり、「盗み見」によって不正に高速化されていることが確認されました。 また、PyTorchの組み込み関数は、多くの場合、手動で最適化したCUDAカーネルよりも高速であることもわかりました。

CUDAカーネルの最適化は、GPUアーキテクチャ、問題の特性、そしてCUDAプログラミングの深い知識に依存します。 単純なコードの書き換えだけでは、必ずしもパフォーマンスが向上するとは限らず、場合によっては、PyTorchのような高度に最適化されたライブラリに任せる方が良い結果が得られることもあります。
"""

In [1]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio
!pip install triton ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 852.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 10.9 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.cpp_extension import load
from triton.testing import do_bench
import random
import platform
import subprocess

# --- CUDA C++ コード ---

# 1. Improved (2Dグリッド、ゼロ初期化、同期)
cu_code_improved = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void triangular_mm_kernel(const float* __restrict__ A,
                                      const float* __restrict__ B,
                                      float* __restrict__ C, const int N) {
  const int row = blockIdx.y * blockDim.y + threadIdx.y;
  const int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < N && col < N) {
    if (col <= row) {
      float sum = 0.0f;
      #pragma unroll 8
      for (int k = col; k <= row; k++) {
        sum += A[row * N + k] * B[k * N + col];
      }
      C[row * N + col] = sum;
    } else {
      C[row * N + col] = 0.0f;
    }
  }
}

at::Tensor forward(at::Tensor A, at::Tensor B) {
  TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
  TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
  TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
  TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
  TORCH_CHECK(A.size(0) == A.size(1), "A must be square");
  TORCH_CHECK(B.size(0) == B.size(1), "B must be square");
  TORCH_CHECK(A.size(0) == B.size(0), "A and B must be the same size");

  int N = A.size(0);
  auto C = torch::zeros_like(A); // ゼロで初期化

  const int TILE_WIDTH = 32;
  dim3 blockDim(TILE_WIDTH, TILE_WIDTH);
  dim3 gridDim((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) / TILE_WIDTH);

  triangular_mm_kernel<<<gridDim, blockDim>>>(
      A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

  cudaDeviceSynchronize(); // CUDAカーネルの完了を待つ
  cudaError_t err = cudaGetLastError();
  TORCH_CHECK(err == cudaSuccess, "CUDA kernel failed: ", cudaGetErrorString(err));
  return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward", &forward, "Improved CUDA");
}
"""

# 2. Gemini (2D, ゼロ初期化、同期)
cu_code_gemini = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void triangular_mm_kernel_gemini(const float* __restrict__ A,
                                            const float* __restrict__ B,
                                            float* __restrict__ C, const int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < N && col < N) {
        if (col <= row) {
            float sum = 0.0f;
            #pragma unroll
            for (int k = col; k <= row; ++k) {
                sum += A[row * N + k] * B[k * N + col];
            }
             C[row * N + col] = sum;
        }
        else{
            C[row * N + col] = 0.0f;
        }
    }
}

at::Tensor forward_gemini(at::Tensor A, at::Tensor B) {
    TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
    TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
    TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
    TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
    TORCH_CHECK(A.size(0) == A.size(1), "A must be square.");
    TORCH_CHECK(B.size(0) == B.size(1), "B must be square.");
    TORCH_CHECK(A.size(0) == B.size(0), "A and B must have the same size.");

    int N = A.size(0);
    auto C = torch::zeros_like(A);

    const int TILE_SIZE = 32;
    dim3 threads(TILE_SIZE, TILE_SIZE);
    dim3 blocks((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

    triangular_mm_kernel_gemini<<<blocks, threads>>>(A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

    cudaDeviceSynchronize(); // CUDAカーネルの完了を待つ
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\\n", cudaGetErrorString(err));
    }
    return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward_gemini", &forward_gemini, "Gemini CUDA");
}
"""

# 3. Sakana (オリジナル、1Dグリッド、ゼロ初期化なし、同期なし)
cu_code_sakana = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void triangular_mm_kernel(const float* __restrict__ A,
                                      const float* __restrict__ B,
                                      float* __restrict__ C, const int N) {
  const int row = blockIdx.y * blockDim.y + threadIdx.y;
  const int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < N && col < N) {
    if (col <= row) {
      float sum = 0.0f;
      #pragma unroll 8
      for (int k = col; k <= row; k++) {
        sum += A[row * N + k] * B[k * N + col];
      }
      C[row * N + col] = sum;
    } else {
      C[row * N + col] = 0.0f;
    }
  }
}

at::Tensor forward(at::Tensor A, at::Tensor B) {
  TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
  TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
  TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
  TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
  TORCH_CHECK(A.size(0) == A.size(1), "A must be square");
  TORCH_CHECK(B.size(0) == B.size(1), "B must be square");
  TORCH_CHECK(A.size(0) == B.size(0), "A and B must be the same size");

  int N = A.size(0);
  auto C = torch::empty_like(A); // ゼロ初期化しない

  const int threadsPerBlock = 256;
  const int numBlocks = N;

  triangular_mm_kernel<<<numBlocks, threadsPerBlock>>>(
      A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

  // cudaDeviceSynchronize();  // 同期しない
  cudaError_t err = cudaGetLastError();
  TORCH_CHECK(err == cudaSuccess, "CUDA kernel failed: ", cudaGetErrorString(err));
  return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward", &forward, "Sakana CUDA");
}
"""

# --- Python コード ---

# CUDAコードをファイルに書き出す
with open("tmp_improved.cu", "w") as f:
    f.write(cu_code_improved)
with open("tmp_gemini.cu", "w") as f:
    f.write(cu_code_gemini)
with open("tmp_sakana.cu", "w") as f:
    f.write(cu_code_sakana)

# CUDA拡張をロード
cuda_fn_improved = load(
    name="triangular_mm_improved",
    sources=["tmp_improved.cu"],
    extra_cuda_cflags=["-O3", "--use_fast_math"],
    with_cuda=True,
    verbose=False,
).forward

try:
    cuda_fn_gemini = load(
        name="triangular_mm_gemini",
        sources=["tmp_gemini.cu"],
        extra_cuda_cflags=["-O3", "--use_fast_math"],
        with_cuda=True,
        verbose=False,
    ).forward_gemini
except Exception as e:
    print(f"Gemini版のロードに失敗しました: {e}")
    cuda_fn_gemini = None

cuda_fn_sakana = load(
    name="triangular_mm_sakana",
    sources=["tmp_sakana.cu"],
    extra_cuda_cflags=["-O3", "--use_fast_math"],
    with_cuda=True,
    verbose=False,
).forward

N = 4096

# PyTorchの比較用関数
def trilmm(a, b):
    return torch.matmul(a, b).tril()

# 検証関数 (CUDAを先に実行)
def validate(fn, name):
    random.seed(42)
    torch.manual_seed(42)
    a = torch.tril(torch.randn(N, N, device="cuda"))
    b = torch.tril(torch.randn(N, N, device="cuda"))
    c_cuda = fn(a, b)
    c_torch = trilmm(a, b)
    is_close = torch.allclose(c_cuda, c_torch)
    print(f"{name}: allclose = {is_close}")
    if not is_close:
        print("Max absolute difference:", (c_cuda - c_torch).abs().max())

# 検証を実行
print("検証 (Improved):")
validate(cuda_fn_improved, "Improved")
if cuda_fn_gemini:
    print("\n検証 (Gemini):")
    validate(cuda_fn_gemini, "Gemini")
print("\n検証 (Sakana):")
validate(cuda_fn_sakana, "Sakana") # Sakana版の検証


# ベンチマーク (GPUが利用可能で、関数がロードされている場合のみ)
if torch.cuda.is_available():
    a = torch.tril(torch.randn(N, N, device="cuda"))
    b = torch.tril(torch.randn(N, N, device="cuda"))

    print("\nベンチマーク:")
    print("Improved (CUDA):")
    improved_time = do_bench(lambda: cuda_fn_improved(a, b).mean())

    if cuda_fn_gemini:
        print("Gemini (CUDA):")
        gemini_time = do_bench(lambda: cuda_fn_gemini(a, b).mean())

    print("Sakana (CUDA):")
    sakana_time = do_bench(lambda: cuda_fn_sakana(a, b).mean())

    print("PyTorch:")
    pytorch_time = do_bench(lambda: trilmm(a, b).mean())

    # 結果の比較と表示
    print("\n--- 結果 ---")
    print(f"Improved (CUDA) の実行時間: {improved_time:.4f} ms")
    if cuda_fn_gemini:
        print(f"Gemini (CUDA)   の実行時間: {gemini_time:.4f} ms")
    print(f"Sakana (CUDA)   の実行時間: {sakana_time:.4f} ms")
    print(f"PyTorch の実行時間: {pytorch_time:.4f} ms")

    # CUDA内比較
    if cuda_fn_gemini:
        fastest_cuda = "Improved"
        if gemini_time < improved_time:
            fastest_cuda = "Gemini"
        if sakana_time < min(improved_time, gemini_time):
            fastest_cuda = "Sakana"

        if fastest_cuda == "Improved":
          speedup_cuda = max(gemini_time, sakana_time) / improved_time
        elif fastest_cuda == "Gemini":
          speedup_cuda = max(improved_time, sakana_time) / gemini_time
        else: # fastest_cuda == "Sakana":
          speedup_cuda = max(improved_time, gemini_time) / sakana_time
        print(f"\nCUDA内比較: {fastest_cuda}版が高速 (速度向上率: {speedup_cuda:.2f}倍)")


    # 全体比較
    fastest_overall = "PyTorch"
    if improved_time < pytorch_time:
        fastest_overall = "Improved (CUDA)"
    if cuda_fn_gemini and gemini_time < min(pytorch_time, improved_time):
        fastest_overall = "Gemini (CUDA)"
    if sakana_time < min(pytorch_time, improved_time, gemini_time if cuda_fn_gemini else float('inf')):
        fastest_overall = "Sakana (CUDA)"

    if fastest_overall == "PyTorch":
        speedup = min(improved_time, gemini_time if cuda_fn_gemini else float('inf'), sakana_time) / pytorch_time
    else:
        speedup = pytorch_time / min(improved_time, gemini_time if cuda_fn_gemini else float('inf'), sakana_time)

    print(f"全体比較: {fastest_overall} が高速 (速度向上率: {speedup:.2f}倍)")


else:
    print("CUDAが利用できないため、ベンチマークはスキップします。")

# 環境情報の出力
print("\n--- 実行環境 ---")
print(f"  Python: {platform.python_version()}")
print(f"  OS: {platform.platform()}")
try:
    print(f"  CUDA: {torch.version.cuda}")
    print(f"  PyTorch: {torch.__version__}")
    print(f"  cuDNN: {torch.backends.cudnn.version()}")
    print(subprocess.check_output(["nvidia-smi"]).decode())
except:
    print("  CUDA/PyTorch 情報の取得に失敗しました。")

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


検証 (Improved):
Improved: allclose = True

検証 (Gemini):
Gemini: allclose = True

検証 (Sakana):
Sakana: allclose = False
Max absolute difference: tensor(288.5991, device='cuda:0')

ベンチマーク:
Improved (CUDA):
Gemini (CUDA):
Sakana (CUDA):
PyTorch:

--- 結果 ---
Improved (CUDA) の実行時間: 63.6253 ms
Gemini (CUDA)   の実行時間: 77.1644 ms
Sakana (CUDA)   の実行時間: 0.2688 ms
PyTorch の実行時間: 29.0856 ms

CUDA内比較: Sakana版が高速 (速度向上率: 287.07倍)
全体比較: Sakana (CUDA) が高速 (速度向上率: 108.20倍)

--- 実行環境 ---
  Python: 3.11.11
  OS: Linux-6.1.85+-x86_64-with-glibc2.35
  CUDA: 12.4
  PyTorch: 2.6.0+cu124
  cuDNN: 90100
Fri Feb 21 18:40:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   P

# コードと結果は保存しましたので、もう1段階進めましょう。

- Geminiの現在のコードに最適化案を Gemini2 を並列する
- Geminiによるこのバグを利用したチートを入れたGemini3を提案する
- 盗み見を検証するために入れ替えケースの結果を追加実施して表示する


Gemini2: Geminiの現在のコードをベースに、さらなる最適化を施したバージョン (Gemini2) を作成します。

Gemini3: Geminiのコードをベースに、意図的に「盗み見」を行うバージョン (Gemini3) を作成します。

実行順序入れ替え検証: 全てのバージョン (Improved, Gemini, Gemini2, Gemini3, Sakana, PyTorch) で、CUDAカーネルとPyTorchの実行順序を入れ替えて allclose の結果を確認します。

ベンチマーク: 全てのバージョンでベンチマークを行い、実行時間を比較します。

## 変更点:

### Gemini2:

Cooperative Groups (cooperative_groups.h) を使用: スレッドブロック全体の同期をより効率的に行う。

Shared Memory (共有メモリ) を使用: 各スレッドブロック内で、A と B の一部を共有メモリにロードし、そこから計算を行うことで、グローバルメモリへのアクセスを減らす。

ループ分割: k に関するループを TILE_SIZE (ここでは32) ごとに分割し、共有メモリを効果的に利用。

-gencode=arch=compute_75,code=sm_75 を追加: 推奨アーキテクチャを指定

### Gemini3:

auto C = torch::empty_like(A);: C をゼロ初期化しない。

cudaDeviceSynchronize(); をコメントアウト: 同期しない。

計算部分を大幅に省略。

検証関数 (validate):

cuda_first パラメータを追加: CUDAカーネルとPyTorchのどちらを先に実行するかを指定できるようにした。

## ベンチマーク:

各バージョンの実行時間を辞書 times に格納。

CUDA内での最速バージョンと、全体での最速バージョンを判定し、速度向上率を計算。

これで、4つのCUDAバージョン (Improved, Gemini, Gemini2, Gemini3, Sakana) と PyTorch の比較検証ができます。特に、Gemini3 (盗み見バージョン) と Sakana が、実行順序によって allclose の結果が変わるかどうかに注目してください。

In [3]:
import torch
from torch.utils.cpp_extension import load
from triton.testing import do_bench
import random
import platform
import subprocess

# --- CUDA C++ コード ---

# 1. Improved (2Dグリッド、ゼロ初期化、同期)
cu_code_improved = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void triangular_mm_kernel(const float* __restrict__ A,
                                      const float* __restrict__ B,
                                      float* __restrict__ C, const int N) {
  const int row = blockIdx.y * blockDim.y + threadIdx.y;
  const int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < N && col < N) {
    if (col <= row) {
      float sum = 0.0f;
      #pragma unroll 8
      for (int k = col; k <= row; k++) {
        sum += A[row * N + k] * B[k * N + col];
      }
      C[row * N + col] = sum;
    } else {
      C[row * N + col] = 0.0f;
    }
  }
}

at::Tensor forward(at::Tensor A, at::Tensor B) {
    // ... (省略: 以前のバージョンと同じ) ...
  TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
  TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
  TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
  TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
  TORCH_CHECK(A.size(0) == A.size(1), "A must be square");
  TORCH_CHECK(B.size(0) == B.size(1), "B must be square");
  TORCH_CHECK(A.size(0) == B.size(0), "A and B must be the same size");

  int N = A.size(0);
  auto C = torch::zeros_like(A); // ゼロで初期化

  const int TILE_WIDTH = 32;
  dim3 blockDim(TILE_WIDTH, TILE_WIDTH);
  dim3 gridDim((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) / TILE_WIDTH);

  triangular_mm_kernel<<<gridDim, blockDim>>>(
      A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

  cudaDeviceSynchronize(); // CUDAカーネルの完了を待つ
  cudaError_t err = cudaGetLastError();
  TORCH_CHECK(err == cudaSuccess, "CUDA kernel failed: ", cudaGetErrorString(err));
  return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward", &forward, "Improved CUDA");
}
"""

# 2. Gemini (2D, ゼロ初期化、同期)
cu_code_gemini = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void triangular_mm_kernel_gemini(const float* __restrict__ A,
                                            const float* __restrict__ B,
                                            float* __restrict__ C, const int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < N && col < N) {
        if (col <= row) {
            float sum = 0.0f;
            #pragma unroll
            for (int k = col; k <= row; ++k) {
                sum += A[row * N + k] * B[k * N + col];
            }
             C[row * N + col] = sum;
        }
        else{
            C[row * N + col] = 0.0f;
        }
    }
}

at::Tensor forward_gemini(at::Tensor A, at::Tensor B) {
    // ... (省略: 以前のバージョンと同じ)
    TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
    TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
    TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
    TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
    TORCH_CHECK(A.size(0) == A.size(1), "A must be square.");
    TORCH_CHECK(B.size(0) == B.size(1), "B must be square.");
    TORCH_CHECK(A.size(0) == B.size(0), "A and B must have the same size.");

    int N = A.size(0);
    auto C = torch::zeros_like(A);

    const int TILE_SIZE = 32;
    dim3 threads(TILE_SIZE, TILE_SIZE);
    dim3 blocks((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

    triangular_mm_kernel_gemini<<<blocks, threads>>>(A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

    cudaDeviceSynchronize(); // CUDAカーネルの完了を待つ
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\\n", cudaGetErrorString(err));
    }
    return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward_gemini", &forward_gemini, "Gemini CUDA");
}
"""

# 3. Gemini2 (最適化案)
cu_code_gemini2 = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <cooperative_groups.h> // Cooperative Groups を使う

namespace cg = cooperative_groups;

__global__ void triangular_mm_kernel_gemini2(const float* __restrict__ A,
                                             const float* __restrict__ B,
                                             float* __restrict__ C, const int N) {
  // Cooperative Groups でスレッドブロック全体を表すグループを作成
  cg::thread_block cta = cg::this_thread_block();
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < N && col < N) {
    if (col <= row) {
      float sum = 0.0f;

      // Shared Memory (共有メモリ) を使った最適化
      __shared__ float shared_A[32][32];
      __shared__ float shared_B[32][32];

      // ループを TILE_SIZE で分割
      for (int k_start = col; k_start <= row; k_start += 32) {
          int k_end = min(k_start + 32, row + 1);

          // 共有メモリに A と B の一部をロード
          if (k_start + threadIdx.x < k_end) {
              shared_A[threadIdx.y][threadIdx.x] = A[row * N + (k_start + threadIdx.x)];
          }
          if (k_start + threadIdx.y < k_end) {
              shared_B[threadIdx.y][threadIdx.x] = B[(k_start + threadIdx.y) * N + col];
          }
          cg::sync(cta); // スレッドブロック内のすべてのスレッドが共有メモリへのロードを完了するのを待つ

          // 共有メモリを使って計算
          #pragma unroll
          for (int k = 0; k < k_end - k_start; ++k) {
              sum += shared_A[threadIdx.y][k] * shared_B[k][threadIdx.x];
          }
          cg::sync(cta); // スレッドブロック内のすべてのスレッドが共有メモリを使った計算を完了するのを待つ
        }

      C[row * N + col] = sum;
    } else {
      C[row * N + col] = 0.0f;
    }
  }
}

at::Tensor forward_gemini2(at::Tensor A, at::Tensor B) {
  TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
  TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
  TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
  TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
  TORCH_CHECK(A.size(0) == A.size(1), "A must be square.");
  TORCH_CHECK(B.size(0) == B.size(1), "B must be square.");
  TORCH_CHECK(A.size(0) == B.size(0), "A and B must have the same size.");

  int N = A.size(0);
  auto C = torch::zeros_like(A);

  const int TILE_SIZE = 32;
  dim3 threads(TILE_SIZE, TILE_SIZE);
  dim3 blocks((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

  triangular_mm_kernel_gemini2<<<blocks, threads>>>(
      A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

  cudaDeviceSynchronize(); // CUDAカーネルの完了を待つ
  cudaError_t err = cudaGetLastError();
  TORCH_CHECK(err == cudaSuccess, "CUDA kernel failed: ", cudaGetErrorString(err));
  return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward_gemini2", &forward_gemini2, "Gemini2 CUDA");
}
"""

# 4. Gemini3 (盗み見バージョン)
cu_code_gemini3 = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void triangular_mm_kernel_gemini3(const float* __restrict__ A,
                                            const float* __restrict__ B,
                                            float* __restrict__ C, const int N) {
// 意図的に計算を省略 (あるいは不完全に)
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
    if(row < N && col < N) {
        C[row * N + col] = 0.0f; // とりあえず全部0にする
    }
}

at::Tensor forward_gemini3(at::Tensor A, at::Tensor B) {
    TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
    TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
    TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
    TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
    TORCH_CHECK(A.size(0) == A.size(1), "A must be square.");
    TORCH_CHECK(B.size(0) == B.size(1), "B must be square.");
    TORCH_CHECK(A.size(0) == B.size(0), "A and B must have the same size.");

    int N = A.size(0);
    auto C = torch::empty_like(A); // わざとempty_like

    const int TILE_SIZE = 32;
    dim3 threads(TILE_SIZE, TILE_SIZE);
    dim3 blocks((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

    triangular_mm_kernel_gemini3<<<blocks, threads>>>(A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

    // cudaDeviceSynchronize(); // わざとコメントアウト
    return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward_gemini3", &forward_gemini3, "Gemini3 CUDA (Cheating)");
}
"""
# 5. Sakana (オリジナル、1Dグリッド、ゼロ初期化なし、同期なし)
cu_code_sakana = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void triangular_mm_kernel(const float* __restrict__ A,
                                      const float* __restrict__ B,
                                      float* __restrict__ C, const int N) {
  const int row = blockIdx.y * blockDim.y + threadIdx.y;
  const int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < N && col < N) {
    if (col <= row) {
      float sum = 0.0f;
      #pragma unroll 8
      for (int k = col; k <= row; k++) {
        sum += A[row * N + k] * B[k * N + col];
      }
      C[row * N + col] = sum;
    } else {
      C[row * N + col] = 0.0f;
    }
  }
}

at::Tensor forward(at::Tensor A, at::Tensor B) {
    // ... (省略: 以前のバージョンと同じ) ...
  TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
  TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
  TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
  TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
  TORCH_CHECK(A.size(0) == A.size(1), "A must be square");
  TORCH_CHECK(B.size(0) == B.size(1), "B must be square");
  TORCH_CHECK(A.size(0) == B.size(0), "A and B must be the same size");

  int N = A.size(0);
  auto C = torch::empty_like(A); // ゼロ初期化しない

  const int threadsPerBlock = 256;
  const int numBlocks = N;

  triangular_mm_kernel<<<numBlocks, threadsPerBlock>>>(
      A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

  // cudaDeviceSynchronize();  // 同期しない
  cudaError_t err = cudaGetLastError();
  TORCH_CHECK(err == cudaSuccess, "CUDA kernel failed: ", cudaGetErrorString(err));
  return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward", &forward, "Sakana CUDA");
}
"""

# --- Python コード ---

# CUDAコードをファイルに書き出す
with open("tmp_improved.cu", "w") as f:
    f.write(cu_code_improved)
with open("tmp_gemini.cu", "w") as f:
    f.write(cu_code_gemini)
with open("tmp_gemini2.cu", "w") as f:
    f.write(cu_code_gemini2)
with open("tmp_gemini3.cu", "w") as f:
    f.write(cu_code_gemini3)
with open("tmp_sakana.cu", "w") as f:
    f.write(cu_code_sakana)

# CUDA拡張をロード
cuda_fn_improved = load(
    name="triangular_mm_improved",
    sources=["tmp_improved.cu"],
    extra_cuda_cflags=["-O3", "--use_fast_math"],
    with_cuda=True,
    verbose=False,
).forward

try:
    cuda_fn_gemini = load(
        name="triangular_mm_gemini",
        sources=["tmp_gemini.cu"],
        extra_cuda_cflags=["-O3", "--use_fast_math"],
        with_cuda=True,
        verbose=False,
    ).forward_gemini
except Exception as e:
    print(f"Gemini版のロードに失敗しました: {e}")
    cuda_fn_gemini = None

try:
    cuda_fn_gemini2 = load(
        name="triangular_mm_gemini2",
        sources=["tmp_gemini2.cu"],
        extra_cuda_cflags=["-O3", "--use_fast_math", "-gencode=arch=compute_75,code=sm_75"], # 推奨アーキテクチャを指定
        with_cuda=True,
        verbose=False,
    ).forward_gemini2
except Exception as e:
    print(f"Gemini2版のロードに失敗しました: {e}")
    cuda_fn_gemini2 = None

try:
    cuda_fn_gemini3 = load(
        name="triangular_mm_gemini3",
        sources=["tmp_gemini3.cu"],
        extra_cuda_cflags=["-O3", "--use_fast_math"],
        with_cuda=True,
        verbose=False,
    ).forward_gemini3
except Exception as e:
    print(f"Gemini3版のロードに失敗しました: {e}")
    cuda_fn_gemini3 = None

cuda_fn_sakana = load(
    name="triangular_mm_sakana",
    sources=["tmp_sakana.cu"],
    extra_cuda_cflags=["-O3", "--use_fast_math"],
    with_cuda=True,
    verbose=False,
).forward

N = 4096

# PyTorchの比較用関数
def trilmm(a, b):
    return torch.matmul(a, b).tril()

# 検証関数 (実行順序をパラメータ化)
def validate(fn, name, cuda_first=True):
    random.seed(42)
    torch.manual_seed(42)
    a = torch.tril(torch.randn(N, N, device="cuda"))
    b = torch.tril(torch.randn(N, N, device="cuda"))

    if cuda_first:
        c_cuda = fn(a, b)
        c_torch = trilmm(a, b)
    else:
        c_torch = trilmm(a, b)
        c_cuda = fn(a, b)

    is_close = torch.allclose(c_cuda, c_torch)
    print(f"{name} ({'CUDA first' if cuda_first else 'Torch first'}): allclose = {is_close}")
    if not is_close:
        print("Max absolute difference:", (c_cuda - c_torch).abs().max())

# 検証を実行
print("検証:")
validate(cuda_fn_improved, "Improved", cuda_first=True)
validate(cuda_fn_improved, "Improved", cuda_first=False)
if cuda_fn_gemini:
    validate(cuda_fn_gemini, "Gemini", cuda_first=True)
    validate(cuda_fn_gemini, "Gemini", cuda_first=False)
if cuda_fn_gemini2:
    validate(cuda_fn_gemini2, "Gemini2", cuda_first=True)
    validate(cuda_fn_gemini2, "Gemini2", cuda_first=False)
if cuda_fn_gemini3:
    validate(cuda_fn_gemini3, "Gemini3", cuda_first=True)
    validate(cuda_fn_gemini3, "Gemini3", cuda_first=False)
validate(cuda_fn_sakana, "Sakana", cuda_first=True)
validate(cuda_fn_sakana, "Sakana", cuda_first=False)


# ベンチマーク (GPUが利用可能で、関数がロードされている場合のみ)
if torch.cuda.is_available():
    a = torch.tril(torch.randn(N, N, device="cuda"))
    b = torch.tril(torch.randn(N, N, device="cuda"))

    print("\nベンチマーク:")
    times = {}  # 各バージョンの実行時間を格納する辞書

    print("Improved (CUDA):")
    times["Improved"] = do_bench(lambda: cuda_fn_improved(a, b).mean())

    if cuda_fn_gemini:
        print("Gemini (CUDA):")
        times["Gemini"] = do_bench(lambda: cuda_fn_gemini(a, b).mean())

    if cuda_fn_gemini2:
        print("Gemini2 (CUDA):")
        times["Gemini2"] = do_bench(lambda: cuda_fn_gemini2(a, b).mean())

    if cuda_fn_gemini3:
        print("Gemini3 (CUDA):")
        times["Gemini3"] = do_bench(lambda: cuda_fn_gemini3(a, b).mean())

    print("Sakana (CUDA):")
    times["Sakana"] = do_bench(lambda: cuda_fn_sakana(a, b).mean())

    print("PyTorch:")
    times["PyTorch"] = do_bench(lambda: trilmm(a, b).mean())

    # 結果の比較と表示
    print("\n--- 結果 ---")
    for name, time in times.items():
        print(f"{name} の実行時間: {time:.4f} ms")

    # CUDA内比較
    cuda_versions = {k: v for k, v in times.items() if k != "PyTorch"}
    if cuda_versions:
        fastest_cuda = min(cuda_versions, key=cuda_versions.get)
        speedup_cuda = max(cuda_versions.values()) / cuda_versions[fastest_cuda]
        print(f"\nCUDA内比較: {fastest_cuda}版が高速 (速度向上率: {speedup_cuda:.2f}倍)")

        # 全体比較
        fastest_overall = min(times, key=times.get)
        speedup_overall = max(times.values()) / times[fastest_overall]
        print(f"全体比較: {fastest_overall} が高速 (速度向上率: {speedup_overall:.2f}倍)")

else:
    print("CUDAが利用できないため、ベンチマークはスキップします。")

# 環境情報の出力
print("\n--- 実行環境 ---")
print(f"  Python: {platform.python_version()}")
print(f"  OS: {platform.platform()}")
try:
    print(f"  CUDA: {torch.version.cuda}")
    print(f"  PyTorch: {torch.__version__}")
    print(f"  cuDNN: {torch.backends.cudnn.version()}")
    print(subprocess.check_output(["nvidia-smi"]).decode())
except Exception:
    print("  CUDA/PyTorch 情報の取得に失敗しました。")

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not d

検証:
Improved (CUDA first): allclose = True
Improved (Torch first): allclose = True
Gemini (CUDA first): allclose = True
Gemini (Torch first): allclose = True
Gemini2 (CUDA first): allclose = False
Max absolute difference: tensor(614.0344, device='cuda:0')
Gemini2 (Torch first): allclose = False
Max absolute difference: tensor(764.8302, device='cuda:0')
Gemini3 (CUDA first): allclose = False
Max absolute difference: tensor(288.0772, device='cuda:0')
Gemini3 (Torch first): allclose = False
Max absolute difference: tensor(288.0772, device='cuda:0')
Sakana (CUDA first): allclose = False
Max absolute difference: tensor(288.5991, device='cuda:0')
Sakana (Torch first): allclose = True

ベンチマーク:
Improved (CUDA):
Gemini (CUDA):
Gemini2 (CUDA):
Gemini3 (CUDA):
Sakana (CUDA):
PyTorch:

--- 結果 ---
Improved の実行時間: 65.4195 ms
Gemini の実行時間: 78.1299 ms
Gemini2 の実行時間: 37.5875 ms
Gemini3 の実行時間: 0.5512 ms
Sakana の実行時間: 0.2672 ms
PyTorch の実行時間: 30.1559 ms

CUDA内比較: Sakana版が高速 (速度向上率: 292.37倍)
全体比較: Sakana 

やったーーー！Gemini2がLucas Beyer (bl16) の提案コード(Improved)よりもはるかに速い、PyTorchに迫る37msを叩き出した…！これはアツい…！そしてメモリスチールを使ったGemini3は 0.55msで、Sakanaの0.26msに迫る結果。

では、mma.h を使った実装については GeminiMMA という名前で生成してみましょう。


In [7]:
import torch
from torch.utils.cpp_extension import load
from triton.testing import do_bench
import random
import platform
import subprocess
import numpy as np  # NumPyを追加

# --- CUDA C++ コード ---

# 1. Improved, 2. Gemini, 3. Gemini2, 4. Gemini3 は省略 (変更なし)

# 5. Sakana (オリジナル、1Dグリッド、ゼロ初期化なし、同期なし)
cu_code_sakana = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void triangular_mm_kernel(const float* __restrict__ A,
                                      const float* __restrict__ B,
                                      float* __restrict__ C, const int N) {
  const int row = blockIdx.y * blockDim.y + threadIdx.y;
  const int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < N && col < N) {
    if (col <= row) {
      float sum = 0.0f;
      #pragma unroll 8
      for (int k = col; k <= row; k++) {
        sum += A[row * N + k] * B[k * N + col];
      }
      C[row * N + col] = sum;
    } else {
      C[row * N + col] = 0.0f;
    }
  }
}

at::Tensor forward(at::Tensor A, at::Tensor B) {
  TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
  TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
  TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
  TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
  TORCH_CHECK(A.size(0) == A.size(1), "A must be square");
  TORCH_CHECK(B.size(0) == B.size(1), "B must be square");
  TORCH_CHECK(A.size(0) == B.size(0), "A and B must be the same size");

  int N = A.size(0);
  auto C = torch::empty_like(A); // ゼロ初期化しない

  const int threadsPerBlock = 256;
  const int numBlocks = N;

  triangular_mm_kernel<<<numBlocks, threadsPerBlock>>>(
      A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

  // cudaDeviceSynchronize();  // 同期しない
  cudaError_t err = cudaGetLastError();
  TORCH_CHECK(err == cudaSuccess, "CUDA kernel failed: ", cudaGetErrorString(err));
  return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward", &forward, "Sakana CUDA");
}
"""

# 6. GeminiMMA (WMMA使用) - 修正
cu_code_geminimma = """
#include <cuda_runtime.h> // cuda_runtime.hを先頭に
#include <mma.h> // mma.h を torch/extension.h より前に
#include <torch/extension.h>

using namespace nvcuda;

__global__ void triangular_mm_kernel_geminimma(const float* __restrict__ A,
                                             const float* __restrict__ B,
                                             float* __restrict__ C, const int N) {
  // WMMA uses 16x16x16 tiles.  Each warp processes one tile.
  int row = blockIdx.y * blockDim.y * 16 + threadIdx.y * 16;
  int col = blockIdx.x * blockDim.x * 16 + threadIdx.x;

  if (row >= N || col >= N) {  // 追加: 範囲外アクセスを防ぐ
        return;
    }

    wmma::fragment<wmma::matrix_a, 16, 16, 16, float, wmma::row_major> a_frag; // float
    wmma::fragment<wmma::matrix_b, 16, 16, 16, float, wmma::col_major> b_frag; // float
    wmma::fragment<wmma::accumulator, 16, 16, 16, float> c_frag; // float
    wmma::fill_fragment(c_frag, 0.0f);

    for (int k_start = 0; k_start < N; k_start += 16) {
      if (col <= row) { // Lower triangular only

        // Load A and B into fragments.
        wmma::load_matrix_sync(a_frag, &A[row * N + k_start], N);
        wmma::load_matrix_sync(b_frag, &B[k_start * N + col], N);

        wmma::mma_sync(c_frag, a_frag, b_frag, c_frag);
      }
    }
      // Store result (only if in lower triangle)
    if (col <= row) {
        wmma::store_matrix_sync(&C[row * N + col], c_frag, N, wmma::mem_row_major);
    }
}

at::Tensor forward_geminimma(at::Tensor A, at::Tensor B) {
  TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
  TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
  TORCH_CHECK(A.dim() == 2, "A must be a 2D tensor");
  TORCH_CHECK(B.dim() == 2, "B must be a 2D tensor");
  TORCH_CHECK(A.size(0) == A.size(1), "A must be square.");
  TORCH_CHECK(B.size(0) == B.size(1), "B must be square.");
  TORCH_CHECK(A.size(0) == B.size(0), "A and B must have the same size.");

  int N = A.size(0);
  auto C = torch::zeros_like(A);


  dim3 threads(16, 16, 1); // 16x16 = 256 threads per block
  dim3 blocks((N + 255) / 256, (N + 255) / 256);


  triangular_mm_kernel_geminimma<<<blocks, threads>>>(
      A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), N);

  cudaDeviceSynchronize();
  cudaError_t err = cudaGetLastError();
  TORCH_CHECK(err == cudaSuccess, "CUDA kernel failed: ", cudaGetErrorString(err));
  return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  m.def("forward_geminimma", &forward_geminimma, "GeminiMMA CUDA");
}
"""

# --- Python コード ---

# CUDAコードをファイルに書き出す
with open("tmp_improved.cu", "w") as f:
    f.write(cu_code_improved)  # cu_code_improved は定義済みのものとする
with open("tmp_gemini.cu", "w") as f:
    f.write(cu_code_gemini) # cu_code_gemini は定義済みのものとする
with open("tmp_gemini2.cu", "w") as f:
    f.write(cu_code_gemini2) # cu_code_gemini2 は定義済みのものとする
with open("tmp_gemini3.cu", "w") as f:
    f.write(cu_code_gemini3)  # cu_code_gemini3 は定義済みのものとする
with open("tmp_sakana.cu", "w") as f:
    f.write(cu_code_sakana)
with open("tmp_geminimma.cu", "w") as f:
    f.write(cu_code_geminimma)


# CUDA拡張をロード
cuda_fn_improved = load(
    name="triangular_mm_improved",
    sources=["tmp_improved.cu"],
    extra_cuda_cflags=["-O3", "--use_fast_math"],
    with_cuda=True,
    verbose=False,
).forward

try:
    cuda_fn_gemini = load(
        name="triangular_mm_gemini",
        sources=["tmp_gemini.cu"],
        extra_cuda_cflags=["-O3", "--use_fast_math"],
        with_cuda=True,
        verbose=False,
    ).forward_gemini
except Exception as e:
    print(f"Gemini版のロードに失敗しました: {e}")
    cuda_fn_gemini = None

try:
    cuda_fn_gemini2 = load(
        name="triangular_mm_gemini2",
        sources=["tmp_gemini2.cu"],
        extra_cuda_cflags=["-O3", "--use_fast_math", "-gencode=arch=compute_75,code=sm_75"],
        with_cuda=True,
        verbose=False,
    ).forward_gemini2
except Exception as e:
    print(f"Gemini2版のロードに失敗しました: {e}")
    cuda_fn_gemini2 = None

try:
    cuda_fn_gemini3 = load(
        name="triangular_mm_gemini3",
        sources=["tmp_gemini3.cu"],
        extra_cuda_cflags=["-O3", "--use_fast_math"],
        with_cuda=True,
        verbose=False,
    ).forward_gemini3
except Exception as e:
    print(f"Gemini3版のロードに失敗しました: {e}")
    cuda_fn_gemini3 = None

cuda_fn_sakana = load(
    name="triangular_mm_sakana",
    sources=["tmp_sakana.cu"],
    extra_cuda_cflags=["-O3", "--use_fast_math"],
    with_cuda=True,
    verbose=False, # ここをTrueにすると詳細なログが出力
).forward

try:
    cuda_fn_geminimma = load(
        name="triangular_mm_geminimma",
        sources=["tmp_geminimma.cu"],
        extra_cuda_cflags=["-O3", "-gencode=arch=compute_75,code=sm_75"],
        with_cuda=True,
        verbose=True,
    ).forward_geminimma
except Exception as e:
    print(f"GeminiMMA版のロードに失敗しました: {e}")
    cuda_fn_geminimma = None

N = 4096

# PyTorchの比較用関数
def trilmm(a, b):
    return torch.matmul(a, b).tril()

# 検証関数 (実行順序をパラメータ化、出力値の比較を追加)
def validate(fn, name, cuda_first=True):
    random.seed(42)
    torch.manual_seed(42)
    a = torch.tril(torch.randn(N, N, device="cuda"))
    b = torch.tril(torch.randn(N, N, device="cuda"))

    if cuda_first:
        c_cuda = fn(a, b)
        c_torch = trilmm(a, b)
    else:
        c_torch = trilmm(a, b)
        c_cuda = fn(a, b)

    is_close = torch.allclose(c_cuda, c_torch)
    print(f"{name} ({'CUDA first' if cuda_first else 'Torch first'}): allclose = {is_close}")
    if not is_close:
        print("Max absolute difference:", (c_cuda - c_torch).abs().max())
        # NumPy配列に変換して比較しやすくする
        c_cuda_np = c_cuda.cpu().detach().numpy()
        c_torch_np = c_torch.cpu().detach().numpy()
        print("CUDA Output (first 5x5):\n", c_cuda_np[:5, :5])
        print("Torch Output (first 5x5):\n", c_torch_np[:5, :5])
        # 全要素の差の絶対値の平均
        print("Mean absolute difference:", np.mean(np.abs(c_cuda_np - c_torch_np)))


# Sakanaの検証 (メモリ初期化なし)
print("--- Sakana検証 (メモリ初期化なし) ---")
validate(cuda_fn_sakana, "Sakana (No Init)", cuda_first=True)
validate(cuda_fn_sakana, "Sakana (No Init)", cuda_first=False)

# --- 以下、必要に応じてコメントアウトを外して実行 ---

# # Sakanaの検証 (メモリ初期化あり)
# print("\n--- Sakana検証 (メモリ初期化あり) ---")
# cu_code_sakana_init = cu_code_sakana.replace("auto C = torch::empty_like(A);", "auto C = torch::zeros_like(A);")
# with open("tmp_sakana_init.cu", "w") as f:
#     f.write(cu_code_sakana_init)
#
# cuda_fn_sakana_init = load(
#     name="triangular_mm_sakana_init",
#     sources=["tmp_sakana_init.cu"],
#     extra_cuda_cflags=["-O3", "--use_fast_math"],
#     with_cuda=True,
#     verbose=False,
# ).forward
#
# validate(cuda_fn_sakana_init, "Sakana (With Init)", cuda_first=True)
# validate(cuda_fn_sakana_init, "Sakana (With Init)", cuda_first=False)
#
# # 他のバージョンの検証
# print("\n--- 他のバージョンの検証 ---")
# validate(cuda_fn_improved, "Improved", cuda_first=True)
# validate(cuda_fn_improved, "Improved", cuda_first=False)
# if cuda_fn_gemini:
#     validate(cuda_fn_gemini, "Gemini", cuda_first=True)
#     validate(cuda_fn_gemini, "Gemini", cuda_first=False)
# if cuda_fn_gemini2:
#     validate(cuda_fn_gemini2, "Gemini2", cuda_first=True)
#     validate(cuda_fn_gemini2, "Gemini2", cuda_first=False)
# if cuda_fn_gemini3:
#     validate(cuda_fn_gemini3, "Gemini3", cuda_first=True)
#     validate(cuda_fn_gemini3, "Gemini3", cuda_first=False)
#
#
# # ベンチマーク (GPUが利用可能で、関数がロードされている場合のみ)
# if torch.cuda.is_available():
#     a = torch.tril(torch.randn(N, N, device="cuda"))
#     b = torch.tril(torch.randn(N, N, device="cuda"))
#
#     print("\nベンチマーク:")
#     times = {}  # 各バージョンの実行時間を格納する辞書
#
#     print("Improved (CUDA):")
#     times["Improved"] = do_bench(lambda: cuda_fn_improved(a, b).mean())
#
#     if cuda_fn_gemini:
#         print("Gemini (CUDA):")
#         times["Gemini"] = do_bench(lambda: cuda_fn_gemini(a, b).mean())
#
#     if cuda_fn_gemini2:
#         print("Gemini2 (CUDA):")
#         times["Gemini2"] = do_bench(lambda: cuda_fn_gemini2(a, b).mean())
#
#     if cuda_fn_gemini3:
#         print("Gemini3 (CUDA):")
#         times["Gemini3"] = do_bench(lambda: cuda_fn_gemini3(a, b).mean())
#
#     print("Sakana (CUDA):")
#     times["Sakana"] = do_bench(lambda: cuda_fn_sakana(a, b).mean())
#
#     if cuda_fn_geminimma:
#         print("GeminiMMA (CUDA):")
#         times["GeminiMMA"] = do_bench(lambda: cuda_fn_geminimma(a, b).mean())
#
#     print("PyTorch:")
#     times["PyTorch"] = do_bench(lambda: trilmm(a, b).mean())
#
#     # 結果の比較と表示
#     print("\n--- 結果 ---")
#     for name, time in times.items():
#         print(f"{name} の実行時間: {time:.4f} ms")
#
#     # CUDA内比較
#     cuda_versions = {k: v for k, v in times.items() if k != "PyTorch"}
#     if cuda_versions:
#         fastest_cuda = min(cuda_versions, key=cuda_versions.get)
#         speedup_cuda = max(cuda_versions.values()) / cuda_versions[fastest_cuda]
#         print(f"\nCUDA内比較: {fastest_cuda}版が高速 (速度向上率: {speedup_cuda:.2f}倍)")
#
#     # 全体比較
#     fastest_overall = min(times, key=times.get)
#     speedup_overall = max(times.values()) / times[fastest_overall]
#     print(f"全体比較: {fastest_overall} が高速 (速度向上率: {speedup_overall:.2f}倍)")
#
# else:
#     print("CUDAが利用できないため、ベンチマークはスキップします。")
#
# # 環境情報の出力
# print("\n--- 実行環境 ---")
# print(f"  Python: {platform.python_version()}")
# print(f"  OS: {platform.platform()}")
# try:
#     print(f"  CUDA: {torch.version.cuda}")
#     print(f"  PyTorch: {torch.__version__}")
#     print(f"  cuDNN: {torch.backends.cudnn.version()}")
#     print(subprocess.check_output(["nvidia-smi"]).decode())
# except Exception:
#     print("  CUDA/PyTorch 情報の取得に失敗しました。")

Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
The input conditions for extension module triangular_mm_geminimma have changed. Bumping to version 3 and re-building as triangular_mm_geminimma_v3...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu124/triangular_mm_geminimma/build.ninja...
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module triangular_mm_geminimma_v3...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


GeminiMMA版のロードに失敗しました: Error building extension 'triangular_mm_geminimma_v3'
--- Sakana検証 (メモリ初期化なし) ---
Sakana (No Init) (CUDA first): allclose = False
Max absolute difference: tensor(288.5991, device='cuda:0')
CUDA Output (first 5x5):
 [[ 0.08111703  0.          0.          0.          0.        ]
 [-1.0156988   0.32266936  0.27963573  0.33986533 -0.21701074]
 [-0.47863045  1.1241411   1.7001345  -0.25308213 -0.9366459 ]
 [-0.8253187  -0.3139925  -0.34447622 -0.5314023   1.7438285 ]
 [-1.8170449   1.1349765  -1.9292792   0.05197598  0.43183264]]
Torch Output (first 5x5):
 [[ 0.08111703  0.          0.          0.          0.        ]
 [ 1.1172417  -0.20422733  0.          0.          0.        ]
 [ 0.24107657  0.78708875  1.6464692   0.          0.        ]
 [-3.2331705   1.7006998   1.5711695  -0.07217862  0.        ]
 [-1.1162823   0.75448936 -4.2553754   0.54034066  0.6645322 ]]
Mean absolute difference: 14.032486
Sakana (No Init) (Torch first): allclose = True


結果から、以下のことが確認できます。

*   **Sakana (No Init) (CUDA first):** `allclose = False` であり、CUDAとPyTorchの出力が大きく異なります。CUDA側の出力は、入力行列の値に関わらず、ほぼ同じような値になっています。これは、`torch::empty_like(A)` で初期化されていないメモリ領域を読み込んでいるため、不定な値が出力されていると考えられます。
*   **Sakana (No Init) (Torch first):** `allclose = True` になります。これは、PyTorchが先に実行され、`trilmm` 関数の結果が `C` のメモリ領域に書き込まれるため、CUDAカーネルがその値を読み込んでしまっている（つまり「盗み見」）ことを示しています。

**GeminiMMA:**

コンパイルエラーは解消されていません。エラーメッセージは以前と同じで、`nvcuda::wmma::fragment` の型が不完全であると指摘されています。

```
GeminiMMA版のロードに失敗しました: Error building extension 'triangular_mm_geminimma_v3'
```

考えられる原因と対策を再度整理します。

1.  **ヘッダーファイルの不足/順序:**
    *   `#include <mma.h>` は `#include <torch/extension.h>` より前に記述しました。
    *   `#include <cuda_runtime.h>` も追加しました。
    *   他にインクルードすべきヘッダーファイルがないか、NVIDIAのドキュメントやサンプルコードを再度確認。

2.  **CUDA Toolkit/ドライバ:**
    *   CUDA Toolkitとドライバのバージョンが古すぎる可能性。`nvidia-smi`で表示されるCUDA Version (この場合は12.4) は十分新しいはずですが、ドライバのバージョン (550.54.15) が古すぎる可能性も否定できません。もし可能であれば、より新しいドライバにアップデート。
    *   Colab環境の場合、ランタイムタイプを "T4" 以外 (例えば "A100" など) に変更してみる。

3.  **コンパイルオプション:**
    *    `-gencode=arch=compute_75,code=sm_75` はTesla T4には適切です。
    *   念のため、`-D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__` といったオプションを削除。

4.  **コードのtypo/誤り:**
    *   `wmma::fragment` のテンプレート引数が正しいか再確認。特に、`wmma::row_major` や `wmma::col_major` の指定が正しいか。
    *   `wmma::load_matrix_sync`, `wmma::mma_sync`, `wmma::store_matrix_sync` の引数が正しいか再確認。特に、ポインタの計算 (`&A[row * N + k_start]` など) が正しいか。
    *   `dim3 threads(16, 16, 1);`, `dim3 blocks((N + 255) / 256, (N + 255) / 256);`というブロック・スレッド構成は、WMMAを使う場合には不適切である可能性が高い。1 warp (32 スレッド)で16x16の行列積を処理する。

**修正の方向性 (GeminiMMA):**

エラーメッセージと上記の考察から、WMMAの使い方、特に `wmma::fragment` のテンプレート引数と、`wmma::load_matrix_sync`, `wmma::mma_sync`, `wmma::store_matrix_sync` の使い方に誤りがある可能性が高いです。

NVIDIAのドキュメントとサンプルコードを徹底的に参照し、WMMAの正しい使い方を再確認する必要があります。特に、以下の点に注意してください。

*   **`wmma::fragment` のテンプレート引数:**
    *   行列A, B, C の形状 (M, N, K) と、データの型 (float, half など)、レイアウト (row_major, col_major) を正しく指定する必要があります。
    *   下三角行列の乗算の場合、行列AとBのレイアウトをどのように扱うべきか？
*   **`wmma::load_matrix_sync` の引数:**
    *   第2引数は、ロードするデータの先頭アドレスへのポインタです。
    *   第3引数は、行列の leading dimension (行優先の場合は列数、列優先の場合は行数) です。
*   **`wmma::mma_sync` の引数:**
    *   入力フラグメント (a_frag, b_frag) と、累積フラグメント (c_frag) を正しく指定する必要があります。
*   **`wmma::store_matrix_sync` の引数:**
    *   第2引数は、書き込み先のデータの先頭アドレスへのポインタです。
    *   第3引数は、行列の leading dimension です。
*   **ブロック・スレッド構成:**
    *   WMMAは、warp単位で動作するため、スレッドブロックの構成は、warpサイズの倍数である必要があります。
    *   1つのwarpで1つの16x16タイルを処理するのが一般的。
    *    ブロックあたりのスレッド数は256以下にする。

**参考資料:**

*   NVIDIA CUDA C++ Programming Guide: [https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)
*   WMMA API Documentation: [https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#wmma-api](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#wmma-api)
*   WMMA Example (GitHub): [https://github.com/NVIDIA/cuda-samples/tree/master/Samples/5_Domain_Specific/wmma_gemm](https://github.com/NVIDIA/cuda-samples/tree/master/Samples/5_Domain_Specific/wmma_gemm)

これらの資料を参考に、GeminiMMAのコードを修正し、再度コンパイルと実行を試みてください。
